# unzip 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 시드 고정 , device 설정 

In [ ]:
# settings
import pandas as pd
import numpy as np
import torch
import os 
import io
import json
import glob
import random

#GPU 설정
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
print("# device  : ", device)
seed = 77

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(seed)

# device  :  cuda:0


In [ ]:
train = np.load("/content/drive/MyDrive/vision/OCR이미지분류/train_np.npy", allow_pickle = True)
target = np.load("/content/drive/MyDrive/vision/OCR이미지분류/target_encoded_np.npy", allow_pickle = True)

In [ ]:
from PIL import Image
import cv2
import time
import torch
import torchvision
import albumentations
import albumentations.pytorch
from matplotlib import pyplot as plt
import os
import random

# Deit small

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.8 MB/s eta 0:00:00


#### split 

In [ ]:
from transformers import DeiTFeatureExtractor, DeiTForImageClassification, DeiTConfig

In [ ]:
extractor = DeiTFeatureExtractor.from_pretrained('facebook/deit-small-distilled-patch16-224')
# config=DeiTConfig('facebook/deit-small-distilled-patch16-224', num_labels = 159)
model = DeiTForImageClassification.from_pretrained( 'facebook/deit-small-distilled-patch16-224')

Some weights of the model checkpoint at facebook/deit-small-distilled-patch16-224 were not used when initializing DeiTForImageClassification: ['distillation_classifier.bias', 'cls_classifier.weight', 'cls_classifier.bias', 'distillation_classifier.weight']
- This IS expected if you are initializing DeiTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTForImageClassification were not initialized from the model checkpoint at facebook/deit-small-distilled-patch16-224 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream

In [ ]:
model.classifier.out_features = 159
# (classifier): Linear(in_features=384, out_features=1000, bias=True)
print(model)

In [ ]:
class Dataset(torch.utils.data.Dataset) :
  def __init__(self, img_path_np, target, extractor):
    self.imgs = img_path_np 
    self.target = target
    self.extractor  = extractor
  
  def __len__(self):
    return len(self.imgs)

  def __getitem__(self, idx):
    item = {}
    item['img'] =  self.extractor(images= Image.open(self.imgs[idx]).convert('RGB'), return_tensors="pt")
    item['y'] = self.target[idx]  # len 159
    return item

# image collator 
def collate_fn(examples):
  batch = {}
  batch['img'] = torch.stack([x['img']['pixel_values'] for x in examples]).squeeze(1)
  batch['y'] = torch.tensor([x['y'] for x in examples])
  return batch

In [ ]:
out = model(batch['img'].to(device))

In [ ]:
out.logits

tensor([[-0.0957, -2.3004,  0.7848,  ..., -0.9340, -0.5375,  0.7124],
        [-0.3853, -1.9808,  0.3197,  ..., -1.3054, -0.5904,  0.4652],
        [-0.0579, -1.8611,  0.6316,  ..., -0.4733,  0.1519,  0.5960],
        ...,
        [-0.6393, -1.8311, -0.3287,  ..., -0.8843,  0.0606,  0.9284],
        [-0.7407, -1.3543,  0.5581,  ..., -0.3060,  0.1552,  1.0115],
        [-0.6301, -1.1083,  0.0331,  ..., -1.0321,  0.0532,  0.7027]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [ ]:
batch['y']

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
loss_fn(out.logits, batch['y'].to(device))

tensor(7.5355, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
out.logits.to('cpu').detach().numpy()

array([[-0.09567678, -2.3003726 ,  0.78476584, ..., -0.9339962 ,
        -0.53754973,  0.71244264],
       [-0.3853176 , -1.9807636 ,  0.31971675, ..., -1.3054347 ,
        -0.59040964,  0.46519154],
       [-0.05787805, -1.8611178 ,  0.6315767 , ..., -0.47331834,
         0.15185937,  0.59603596],
       ...,
       [-0.6392666 , -1.8310747 , -0.32872403, ..., -0.8843302 ,
         0.06060824,  0.92842394],
       [-0.74071485, -1.3542597 ,  0.5580878 , ..., -0.30599433,
         0.15523499,  1.0114641 ],
       [-0.630101  , -1.108257  ,  0.03310081, ..., -1.0321112 ,
         0.05321133,  0.70274216]], dtype=float32)

In [ ]:
batch['y'].to('cpu').numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# 학습

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in tqdm(dataloader): 
        pred = model(batch["img"].to(device))
        loss = loss_fn(pred.logits, batch["y"].to(device))   
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss


@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 
    
    pred_list = []
    true_list = []
    softmax = torch.nn.Softmax(dim=1) 

    for batch in tqdm(dataloader):   
        pred = model(batch["img"].to(device))
        pred = pred.logits
        
        if batch.get("y") is not None: 
            loss = loss_fn(pred, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred)
        pred = pred.to("cpu").detach().numpy() 
        true = batch['y'].to('cpu').numpy()

        pred_list.append(pred)
        true_list.append(true)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    true = np.concatenate(true_list)
    return epoch_loss , pred , true


In [ ]:
from tqdm import tqdm_notebook
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=3,shuffle=True, random_state=77)
seed_everything(seed)

In [ ]:
seed_everything(77)

# model check ===================================================================
optimizer = torch.optim.RAdam(model.parameters(), lr = 0.00001)
loss_fn = torch.nn.CrossEntropyLoss() 
batch_size = 16

for i,(tri,vai) in enumerate(cv.split(train)):
  if i == 0 : 
      model = model.to(device)
      optimizer = torch.optim.RAdam(model.parameters(),lr=0.00001)
      train_dt = Dataset(train[tri],target[tri], extractor)
      valid_dt = Dataset(train[vai],target[vai], extractor)
      train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn = collate_fn)
      valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False, collate_fn = collate_fn)


      best_score = 0
      patience = 0
      best_score_list = []
      num_epochs = 10
      for epoch in range(num_epochs):
          train_loss = train_loop(train_dl, model , loss_fn, optimizer, device)
          valid_loss , pred , true = test_loop(valid_dl, model , loss_fn,device  )      
          pred = np.argmax(pred, axis=1) 
          score = f1_score(true, pred , average="weighted")
          print(f"train loss {train_loss},  valid loss : {valid_loss} ,  f1-score : {score}")
          patience += 1
          if best_score < score:
              patience = 0 
              best_score = score
              torch.save(model.state_dict(), f"/content/drive/MyDrive/vision/OCR이미지분류/face_book_Levit_fold_{i}_epoch_{epoch}.pth")

          if patience == 3:
              break
          print(f" Epoch ({epoch}), BEST F1: {best_score}")

      print(f"Fold ({i}), BEST F1: {best_score}")
      torch.cuda.empty_cache()
      


100%|██████████| 708/708 [1:34:56<00:00,  8.05s/it]


train loss 1.9078346600303542,  valid loss : 1.634409505684497 ,  f1-score : 0.6783617044885011
 Epoch (0), BEST F1: 0.6783617044885011


100%|██████████| 708/708 [07:29<00:00,  1.57it/s]


train loss 1.155202530745396,  valid loss : 1.0389864811894751 ,  f1-score : 0.7967125220408797
 Epoch (1), BEST F1: 0.7967125220408797


100%|██████████| 708/708 [07:17<00:00,  1.62it/s]


...

train loss 0.04340533082147187,  valid loss : 0.3728456742194018 ,  f1-score : 0.9014249679689155
 Epoch (7), BEST F1: 0.9014249679689155


100%|██████████| 708/708 [07:09<00:00,  1.65it/s]


train loss 0.03176599276093582,  valid loss : 0.33450330716510446 ,  f1-score : 0.9100193969890714
 Epoch (8), BEST F1: 0.9100193969890714


100%|██████████| 708/708 [07:12<00:00,  1.64it/s]


train loss 0.04431139322794911,  valid loss : 0.31774644840556054 ,  f1-score : 0.913931911300326
 Epoch (9), BEST F1: 0.913931911300326
Fold (0), BEST F1: 0.913931911300326
